In [1]:
import os

# change into the folder where your notebook now lives
%cd /Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/BookDB/reranker
print("Working directory is now:", os.getcwd())

/Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/BookDB/reranker
Working directory is now: /Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/BookDB/reranker


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3
from botocore.config import Config
import dask.dataframe as dd

## 1. Load and Prepare Data

In [3]:
import os
import boto3
from botocore.config import Config

def download_from_r2(object_name, local_path, bucket_name="bookdbio"):
    # ensure parent dir exists
    parent_dir = os.path.dirname(local_path)
    if parent_dir and not os.path.isdir(parent_dir):
        os.makedirs(parent_dir, exist_ok=True)

    s3 = boto3.client(
        's3',
        endpoint_url="https://a9a190ee80813000e18bacf626b1281b.r2.cloudflarestorage.com/",
        aws_access_key_id='…',
        aws_secret_access_key='…',
        config=Config(signature_version='s3v4')
    )

    try:
        s3.download_file(bucket_name, object_name, local_path)
        print(f"Successfully downloaded {object_name} to {local_path}")
    except Exception as e:
        print(f"Download failed for {object_name}: {e}")

cwd: /Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/BookDB/reranker


OSError: [Errno 30] Read-only file system: '/mnt'

In [ ]:
download_from_r2("data/books_dedup.parquet", "data/books_dedup.parquet")

In [ ]:
list_bucket_contents()

In [ ]:
# Load interactions data
interactions_df = pd.read_parquet("../data/interactions_prepared_ncf.parquet")

# Split into train/val/test sets
train_df, temp_df = train_test_split(interactions_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
# Initialize reranker
reranker = TwoStageReranker(
    user_embeddings_path="../embeddings/gmf_user_embeddings.parquet",
    book_embeddings_path="../embeddings/gmf_book_embeddings.parquet",
    sbert_embeddings_path="../embeddings/SBERT_embeddings.parquet",
    interactions_path="../data/interactions_prepared_ncf.parquet",
    k_candidates=200,  # Generate 200 candidates
    k_final=10,        # Return top 10 recommendations
    diversity_weight=0.3
)

## 3. Train the Reranker

In [ ]:
# Train the model
reranker.train(train_df, val_df)

## 4. Evaluate the Reranker

In [ ]:
# Evaluate on test set
metrics = reranker.evaluate(test_df)

print("Test Set Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

## 5. Get Recommendations for a User

In [ ]:
# Get a random user ID (implement this method if not present)
random_user = reranker.user_embeddings_df['user_id'].sample(1).iloc[0]
print(f"Random user ID: {random_user}")

# Get recommendations
recommendations = reranker.get_recommendations(random_user)
print("\nRecommended book IDs:")
for i, book_id in enumerate(recommendations, 1):
    print(f"{i}. {book_id}")

In [ ]:
## 6. Feature Importance Analysis

In [ ]:
# Get feature importance from LightGBM model
feature_importance = pd.DataFrame({
    'feature': reranker.model.feature_name(),
    'importance': reranker.model.feature_importance()
})

# Sort by importance
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.bar(feature_importance['feature'], feature_importance['importance'])
plt.xticks(rotation=45, ha='right')
plt.title('Feature Importance in Reranker Model')
plt.tight_layout()
plt.show()